In [1]:
import cupy
import cudf
import pyarrow as pa
import pandas as pd
filepath = "/home/fstrug/uscmshome/nobackup/temp/Run2012B_SingleMu_compressed_zstdlv3_Pv2-6_PPv2-0_PLAIN.parquet"

import pyarrow.parquet as pq
pqfile=pq.ParquetFile(filepath)
print(pqfile.metadata.row_group(0).column(0))

  file_offset: 684
  file_path: 
  physical_type: INT32
  num_values: 308919
  path_in_schema: run
  is_stats_set: True
  statistics:
      has_min_max: True
      min: 194050
      max: 196531
      null_count: 0
      distinct_count: None
      num_values: 308919
      physical_type: INT32
      logical_type: None
      converted_type (legacy): NONE
  compression: ZSTD
  encodings: ('RLE', 'PLAIN')
  has_dictionary_page: False
  dictionary_page_offset: None
  data_page_offset: 4
  total_compressed_size: 680
  total_uncompressed_size: 1235790


In [19]:
%%timeit
# Reading directly to GPU with CUDF
cudf.set_option("copy_on_write", False)
with cupy.cuda.Device() as d:
    MET_pt_cudf_togpu = cudf.read_parquet(filepath, engine="cudf", columns = ["MET_pt"])
    d.synchronize()

121 ms ± 2.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%%timeit
# BROKEN for jagged arrays: Reading to CPU and copying to GPU
with cupy.cuda.Device() as d:
    MET_pt_arrow = pa.parquet.read_table(filepath, columns = ["MET_pt"])
    MET_pt_cudf_cputogpu = cudf.DataFrame.from_arrow(MET_pt_arrow)
    d.synchronize()

190 ms ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
